In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [995 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [915 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,251 kB]
Hit:13 https://ppa.launchpadco

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True)

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("table")

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("SELECT YEAR(date), ROUND(AVG(price),2) as avg_price FROM table WHERE bedrooms = 4 GROUP BY YEAR(date) ORDER BY YEAR(date)")
result.show()

+----------+---------+
|year(date)|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
        SELECT date_built, ROUND(AVG(price), 2) as avg_price
        FROM table
        WHERE bedrooms = 3 AND bathrooms =3
        GROUP BY date_built
        ORDER BY date_built
        """

spark.sql(query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
        SELECT date_built, ROUND(AVG(price)) as avg_price
        FROM table
        WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
        GROUP BY date_built
        ORDER BY date_built
        """
spark.sql(query).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010| 285010.0|
|      2011| 276554.0|
|      2012| 307540.0|
|      2013| 303677.0|
|      2014| 298265.0|
|      2015| 297610.0|
|      2016| 293965.0|
|      2017| 280318.0|
+----------+---------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# I don't think this question makes sense. The price is being aggregated, and you want the view rating for the average prices...
#...but you don't tell us what aggregation to perform on view rating...
#...I think the question needs some revision. There's some information missing here...
# This has been a consistent problem throughout the bootcamp. This feature does not encourage me to recommend the bootcamp.


start_time = time.time()

query = """
        SELECT AVG(view), ROUND(AVG(price))
        FROM table
        WHERE price >= 350000
        """
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------------------+
|avg(view)|round(avg(price), 0)|
+---------+--------------------+
|   32.264|            473796.0|
+---------+--------------------+

--- 1.4845998287200928 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('table')

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('table')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
        SELECT AVG(view), ROUND(AVG(price))
        FROM table
        WHERE price >= 350000
        """
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+---------+--------------------+
|avg(view)|round(avg(price), 0)|
+---------+--------------------+
|   32.264|            473796.0|
+---------+--------------------+

--- 0.510751485824585 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [14]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet("home_sales")
parquet_data

DataFrame[id: string, date: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, date_built: int]

In [15]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [25]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
        SELECT AVG(view), ROUND(AVG(price))
        FROM parquet_table
        WHERE price >= 350000
        """
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+--------------------+
|avg(view)|round(avg(price), 0)|
+---------+--------------------+
|   32.264|            473796.0|
+---------+--------------------+

--- 0.4788484573364258 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('table')

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("table")


False